# Analysis on RTLS
This file is shamelessly taken from the book for the most part

This is taken from the Nolan/Duncan Data Science textbook. I did not write any of it personally

First run the file [/notebooks/CaseStudy1/Clean%20Data%20(From%20Book).ipynb](/notebooks/CaseStudy1/Clean%20Data%20(From%20Book).ipynb)

In [1]:
DATA_FRAME_SOURCE = '../data/offline.final.trace.csv'

In [4]:
offline = read.csv(DATA_FRAME_SOURCE, row.names=1)

In [6]:
head(offline)

time,posX,posY,orientation,mac,signal,rawTime,angle
2006-02-11 01:31:58,0,0,0,00:14:bf:b1:97:8a,-38,1.139643e+12,0
2006-02-11 01:31:58,0,0,0,00:14:bf:b1:97:90,-56,1.139643e+12,0
2006-02-11 01:31:58,0,0,0,00:0f:a3:39:e1:c0,-53,1.139643e+12,0
2006-02-11 01:31:58,0,0,0,00:14:bf:b1:97:8d,-65,1.139643e+12,0
2006-02-11 01:31:58,0,0,0,00:14:bf:b1:97:81,-65,1.139643e+12,0
2006-02-11 01:31:58,0,0,0,00:14:bf:3b:c7:c6,-66,1.139643e+12,0
